### dHBV2 MERIT Download script

This notebook contains code to download the dHBV merit 2 code locally to be used in routing

NOTE: using .nc as an intermediate is necessary since zarr v2 how this data is saved and zarr v3 is used in DDR

Source: https://zenodo.org/records/15784945

In [ ]:
!uv pip install "zarr<3"

In [ ]:
import posixpath

import xarray as xr
import zarr
from tqdm import tqdm

In [ ]:
MERIT_ZONES = ["71", "72", "73", "74", "75", "77", "78"]
datasets = []

for zone in tqdm(MERIT_ZONES, desc="downloading merit zone data"):
    merit_qsim_path = "s3://psu-diff-water-models/dhbv2.0_40yr_dataset/1980-2020_daily_streamflow_merit"
    merit_qsim_path_i = posixpath.join(merit_qsim_path, zone, "1980-01-02_2020-12-30.zarr")
    root = zarr.open_group(merit_qsim_path_i, mode="r", zarr_version=2, storage_options={"anon": True})
    ds = xr.Dataset(
        data_vars={
            "Qr": (["divide_id", "time"], root.Predictions[:]),
        },
        coords={
            "divide_id": (["divide_id"], root.COMID[:]),
            "time": (["time"], root.Time[:]),
        },
    )
    datasets.append(ds)

In [ ]:
# This could take a while depending on the chunk size
combined_ds = xr.concat(datasets, dim="divide_id")
combined_ds

In [ ]:
combined_ds.to_netcdf("./merit_dhbv2_retrospective.nc")